In [1]:
import numpy as np
from numba import jit
from bisect import bisect_right
import quantecon as qe

In [2]:
@jit
def my_interp1(x, xp, fp, left=None, right=None, period=None):
    # Assumes that given xp is increasing
    y = np.empty_like(x)
    for n in range(len(x)):
        if left==None:
            left = fp[0]
        if right==None:
            right = fp[-1]
            
        if x[n] < xp[0]:
            y[n] = left
        elif x[n] > xp[-1]:
            y[n] = right
        else:
            i = bisect_right(xp, x[n])
            y[n] = (fp[i]-fp[i-1])*(x[n]-xp[i-1])/(xp[i]-xp[i-1])+fp[i-1]
    return y

In [3]:
@jit
def my_interp2(x, xp, fp):
    # Assumes that given xp is increasing, and x is not outside range
    bins = np.digitize(x,xp)
    slope = np.diff(fp)/np.diff(xp)
    y = (x-xp[bins-1])*slope[bins-1]+fp[bins-1]
    return y

In [8]:
xp_1 = np.linspace(3, 80, 100)
fp_1 = np.exp(xp_1)
x_1 = np.linspace(4, 60, 20)

In [10]:
%timeit np.interp(x_1,xp_1,fp_1)
%timeit my_interp2(x_1,xp_1,fp_1)
%timeit my_interp1(x_1,xp_1,fp_1) #too slow

The slowest run took 13.46 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.37 µs per loop
100000 loops, best of 3: 4.4 µs per loop
10000 loops, best of 3: 49.5 µs per loop


In [334]:
def coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value and endogenous grid
    c, y = consumption(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

    # Update policy function and return
    Kg = lambda x: my_interp2(x, y, c)
    return Kg

In [356]:
@jit
def consumption(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    c = np.empty_like(k_grid) 
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    y = k_grid+c # y_i = k_i + c_i
    return c, y

In [336]:
def coleman_egm_class(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: np.interp(x, y, c)
    return Kg

In [337]:
## Define the model

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

## Let's make convenience functions based around these primitives

def crra_coleman(g):
    return coleman_egm_class(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

def crra_coleman_egm(g):
    return coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

In [299]:
## Iterate, compare policies

sim_length = 20

print("Timing policy function iteration with endogenous grid without Numba")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman(g)
    g = new_g
qe.util.toc()

print("Timing policy function iteration with endogenous grid and Numba")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm(g)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid without Numba
TOC: Elapsed: 0.19562602043151855 seconds.
Timing policy function iteration with endogenous grid and Numba
TOC: Elapsed: 0.17700409889221191 seconds.


0.17700409889221191